In [ ]:
# installing required module 
!pip install fuzzy-c-means

In [ ]:
import cv2
import numpy as np
import math
import bisect
from google.colab.patches import cv2_imshow
from skimage import morphology
from sklearn.cluster import KMeans
from fcmeans import FCM

def imadjust(src, tol=1, vin=[0,255], vout=(0,255)):
    # src : input one-layer image (numpy array)
    # tol : tolerance, from 0 to 100.
    # vin  : src image bounds
    # vout : dst image bounds
    # return : output img

    dst = src.copy()
    tol = max(0, min(100, tol))

    if tol > 0:
        # Compute in and out limits
        # Histogram
        hist = np.zeros(256, dtype=np.int)
        for r in range(src.shape[0]):
            for c in range(src.shape[1]):
                hist[src[r,c]] += 1
        # Cumulative histogram
        cum = hist.copy()
        for i in range(1, len(hist)):
            cum[i] = cum[i - 1] + hist[i]

        # Compute bounds
        total = src.shape[0] * src.shape[1]
        low_bound = total * tol / 100
        upp_bound = total * (100 - tol) / 100
        vin[0] = bisect.bisect_left(cum, low_bound)
        vin[1] = bisect.bisect_left(cum, upp_bound)

    # Stretching
    scale = (vout[1] - vout[0]) / (vin[1] - vin[0])
    for r in range(dst.shape[0]):
        for c in range(dst.shape[1]):
            vs = max(src[r,c] - vin[0], 0)
            vd = min(int(vs * scale + 0.5) + vout[0], vout[1])
            dst[r,c] = vd
    return dst

def matlab_fspecial(typex = "motion", len = 9, theta = 0):
  # h = fspecial('motion',len,theta)
  if typex == 'motion':
    # Create the vertical kernel. 
    kernel_v = np.zeros((len, theta)) 

    # Fill the middle row with ones. 
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 

    # Normalize. 
    kernel_v /= kernel_size 

    # Apply the vertical kernel. 
    motion_blur = cv2.filter2D(img, -1, kernel_v) 
    return motion_blur

# equal to mat2gray on matlab 
# https://stackoverflow.com/questions/39808545/implement-mat2gray-in-opencv-with-python
def matlab_mat2gray(A, alpha = False, beta = False):
  if not alpha:
    alpha = min(A.flatten())
  else:
    alpha = 0
    
  if not beta:
    beta = max(A.flatten())
  else:
    beta = 255

  I = A
  cv2.normalize(A, I, alpha , beta ,cv2.NORM_MINMAX)
  I = np.uint8(I)
  return I

def matlab_strel_disk(r1):
  from skimage.morphology import disk

  mask = disk(r1)
  return mask

def matlab_strel_ball(r1,r2):
  from skimage.morphology import (octagon)

  mask = octagon(r1,r2)
  return mask

In [ ]:
# function to resize image 
def resize_img(file, size = 200):
  nrows = (np.shape(file)[0])  # image height 
  ncols = (np.shape(file)[1]) 
  ratio = nrows/ncols
  t_width = size
  t_height = math.ceil(t_width * ratio)
  return cv2.resize(file, (t_width, t_height))

# get filename from path
def getfilename(path, ext = False):
  import ntpath
  import os
  if ext:
    return ntpath.basename(path)
  else:
    return os.path.splitext(ntpath.basename(path))[0]

def scanFolder(path = './', max_file_each_folder = "all", verbose = False):
  import os.path # untuk cek file
  
  files_path = []
  in_folder = []
  total_files = str(sum([len(f2)-len(d2) for r2, d2, f2 in os.walk(path)]))
  
  # r=root, d=directories, f = files
  for r, d, f in os.walk(path):
    # cut f
    if max_file_each_folder != "all":
      f = f[:int(max_file_each_folder)]
    
    for file in f:
      if (file.find('.png') > -1 or file.find('.jpg') > -1 or file.find('.tif') > -1):
        files_path.append(os.path.join(r, file))
        in_folder.append(os.path.basename(os.path.dirname(files_path[-1]))) 
  
  if verbose:
    for f, i in zip(files_path, in_folder):
      print(f)
      print(i)

  # ret
  file_scanned = str(len(files_path))

  return files_path, in_folder, total_files, file_scanned
  # // END OF scanFolder

In [ ]:
def dice_coefficient(ground_truth, segmented_img): 

  verbose_mode = False #debug

  # calculating dice 
  nrows = (np.shape(ground_truth)[0])  # image height 
  ncols = (np.shape(ground_truth)[1]) 

  # try agai n
  iGT = ground_truth
  iPM = segmented_img
  iTP = cv2.bitwise_and(iGT, iPM)
  iFN = np.subtract(iGT,iTP)
  iFP = np.subtract(iPM,iTP)
  iTN = cv2.bitwise_not(cv2.bitwise_or(iGT, iPM))

  # sum 
  FN = np.sum(iFN)/255
  FP = np.sum(iFP)/255
  TP = np.sum(iTP)/255
  TN = np.sum(iTN)/255

  # hasil lebih detail
  acc = (TP+TN)/(TP+TN+FP+FN) 
  spec = TN/(TN+FP)
  prec = TP/(TP+FP)
  recall = TP/(TP+FN)
  j = (((TP+TN)/(TP+TN+FP+FN))-((((TP+FN)*(TP+FP))+((TN+FN)*(TN+FP)))/(TP+TN+FP+FN)**2))/(1-((((TP+FN)*(TP+FP))+((TN+FN)*(TN+FP)))/(TP+TN+FP+FN)**2))
  dc = (2*TP)/(2*(TP+FP+FN))

  # hasil dice coef
  import pandas as pd
  from IPython.display import display, HTML

  df = pd.DataFrame(
      {
          "Pred. (1)": [str(TP) + " (TP)", str(FP)  + " (FP)"],
          "Pred. (0)": [str(FN) + " (FN)", str(TN)  + " (TN)"]
      },index=["Actu. (1)", "Actu. (0)"])
  display(HTML(df.to_html()))
  


  df = pd.DataFrame(
      {
          "Accuracy (A)": [acc],
          "Specificity (S)": [spec],
          "Precision (P)": [prec],
          "Recall (R)": [recall],
          "Kappa Index (J)": [j],
          "Dice coefficiet (DC),": [dc],
      })
  display(HTML(df.to_html()))

  # showing image 
  if verbose_mode:
    # show
    print("(a) FN = Ground Truth")
    cv2_imshow(resize_img(iFN,500))
    print("(b) FP = Segmented Image")
    cv2_imshow(resize_img(iFP,500))
    print("(c) TP = Correct Region")
    cv2_imshow(resize_img(iTP,500))
    print("(d) TN = True Negative")
    cv2_imshow(resize_img(iTN,500))
  return df

In [ ]:
# original script of vogado's segmentation method
# clustering using k-means
def wbc_vogado(f, debug_mode = False):
  image_lab = int(0)
  image_rgb = f  # send into figure (a)]

  # time measurement 
  import time
  start_time = time.time()

  # pre-processing step, convert rgb into CIELAB (L*a*b)
  image_lab = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2Lab);
  L = image_lab[:,:,0]
  A = image_lab[:,:,1]
  B = image_lab[:,:,2] # the bcomponent
  lab_y = B # send into figure (c)

  AD = cv2.add(L,B)

  # f (bgr)
  r = f[:,:,2] # red channel (rgb)
  r = imadjust(r)
  g = f[:,:,1]
  g = imadjust(g)
  b = f[:,:,0]
  b = imadjust(b)
  c = np.subtract(255,r)
  c = imadjust(c)
  m = np.subtract(255,g)

  cmyk_m = m # send into figure (b)
  m = cv2.blur(m,(10,10)) # updated in 13/04/2016 - 6:15
  cmyk_m_con_med  = m # send into figure (d)
  m = imadjust(m)
  y = np.subtract(255,b)
  y = imadjust(y)
  AD = matlab_mat2gray(B)
  AD = cv2.medianBlur(AD,7)
  lab_y_con_med = AD # send into figure (e)

  # subtract the M and b
  sub = cv2.subtract(m,AD)
  img_subt = sub #  send into figure (f)
  CMY = np.stack((c,m,y), axis=2) 
  F = np.stack((r, g, b), axis=2)
  ab = CMY # generate CMY color model
  nrows = (np.shape(f)[0])  # image height 
  ncols = (np.shape(f)[1]) 

  # reshape into one single array
  ab = ab.flatten()
  x = nrows
  y = ncols
  data = sub.flatten() # sub = result of subtraction M and b, put them into one long array

  ## step 2 - clustering
  nColors = 3  # Number of clusters (k)
  kmeans = KMeans(n_clusters=nColors, random_state=0)
  kmeans.fit_predict(data.reshape(-1, 1))
  # cluster_idx, cluster_center =  kmeans.cluster_centers_
  cluster_idx = kmeans.labels_ # index result of kmeans
  cluster_center = kmeans.cluster_centers_ #  position of cluster center
  pixel_labels = np.reshape(cluster_idx, (nrows, ncols));
  pixel_labels = np.uint8(pixel_labels)
 
  # the problem is here, 
  tmp =  np.sort(cluster_center.flatten())
  idx = np.zeros((len(tmp), 1))
  for i in range(len(tmp)):
    idx[i] = cluster_center.tolist().index(tmp[i])

  nuclei_cluster = idx[2] # sort asc, nuclei cluster is always who has higher value
  A = np.zeros((nrows, ncols), dtype=np.uint8)

  # print(np.shape(A))
  for row in range(nrows):
    for col in range(ncols):
      # print(" pixel_labels[row,col] = ",  row, col)
      if pixel_labels[row,col] == nuclei_cluster:
        A[row,col] = 255
      else:
        A[row,col] = 0

  ## step 3 - post-processing
  img_clustering = A # send into figure (x)
  img_clustering = imadjust(img_clustering)

  sed = matlab_strel_disk(7) # disk
  see = matlab_strel_ball(3,3) #circle 

  A = cv2.dilate(A,sed)
  # erosion
  A = cv2.erode(A, see)  
  # remove area < 800px 
  A = morphology.area_opening(A, area_threshold=800*3, connectivity=1) # vogado mention he use 800px
  img_morpho = A # send into figure (g)

  # debug mode 
  if(debug_mode):
    # resize image into width 200px 
    ratio = ncols/nrows
    t_width = 200
    t_height = math.ceil(t_width * ratio)

    print("(a) Original")
    cv2_imshow(resize_img(image_rgb, t_width))

    print("(b) M from CMYK")
    cv2_imshow(resize_img(cmyk_m, t_width))

    print("(c) *b from CIELAB")
    cv2_imshow(resize_img(lab_y, t_width))

    print("(d) M con adj + med(7x7)")
    cv2_imshow(resize_img(cmyk_m_con_med, t_width))

    print("(e) *b con adj + med(7x7)")
    cv2_imshow(resize_img(lab_y_con_med, t_width))

    print("(f) *b - M")
    cv2_imshow(resize_img(img_subt, t_width))

    print("(x) clustering" )
    cv2_imshow(resize_img(img_clustering, t_width))

    print("(g)  Morphological Ops.")
    cv2_imshow(resize_img(img_morpho, t_width))

  print("--- %s seconds ---" % (time.time() - start_time))
  return img_morpho

In [ ]:
# modified vogado's segmentaion
# segmentation: fcm
def wbc_vogado_modified(f, debug_mode = False):
  image_lab = int(0)
  image_rgb = f  # send into figure (a)

  # time measurement 
  import time
  start_time = time.time()

  # pre-processing step, convert rgb into CIELAB (L*a*b)
  image_lab = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2Lab);
  L = image_lab[:,:,0]
  A = image_lab[:,:,1]
  B = image_lab[:,:,2] # the bcomponent
  lab_y = B # send into figure (c)

  AD = cv2.add(L,B)

  # f (bgr)
  r = f[:,:,2] # red channel (rgb)
  r = imadjust(r)
  g = f[:,:,1]
  g = imadjust(g)
  b = f[:,:,0]
  b = imadjust(b)
  c = np.subtract(255,r)
  c = imadjust(c)
  m = np.subtract(255,g)

  cmyk_m = m # send into figure (b)

  # add median filter into M component of CMYK
  m = cv2.blur(m,(10,10)) # updated in 13/04/2016 - 6:15
  cmyk_m_con_med  = m # send into figure (d)
  m = imadjust(m)
  y = np.subtract(255,b)
  y = imadjust(y)
  AD = matlab_mat2gray(B)
  AD = cv2.medianBlur(AD,7)
  lab_y_con_med = AD # send into figure (e)
  
  # subtract the M and b
  sub = cv2.subtract(m,AD)
  img_subt = sub #  send into figure (f)
  CMY = np.stack((c,m,y), axis=2) 
  F = np.stack((r, g, b), axis=2)

  ab = CMY # generate CMY color model
  nrows = (np.shape(f)[0])  # image height 
  ncols = (np.shape(f)[1]) 

  # reshape into one single array
  ab = ab.flatten()
  x = nrows
  y = ncols
  data = sub.flatten() # sub = result of subtraction M and b, put them into one long array

  ## step 2 - clustering
  nColors = 3  # Number of clusters (k)

  # fit the fuzzy-c-means
  fcm = FCM(n_clusters=nColors)
  fcm.fit(data.reshape(-1, 1))

  # outputs
  cluster_idx  = fcm.u.argmax(axis=1)
  cluster_center = fcm.centers
  kmeans = fcm

  pixel_labels = np.reshape(cluster_idx, (nrows, ncols));
  pixel_labels = np.uint8(pixel_labels)
 
  # the problem is here, 
  tmp =  np.sort(cluster_center.flatten())
  idx = np.zeros((len(tmp), 1))
  for i in range(len(tmp)):
    idx[i] = cluster_center.tolist().index(tmp[i])
  nuclei_cluster = idx[2] # sort asc, nuclei cluster is always who has higher value
  A = np.zeros((nrows, ncols), dtype=np.uint8)

  # print(np.shape(A))
  for row in range(nrows):
    for col in range(ncols):
      # print(" pixel_labels[row,col] = ",  row, col)
      if pixel_labels[row,col] == nuclei_cluster:
        A[row,col] = 255
      else:
        A[row,col] = 0

  ## step 3 - post-processing
  img_clustering = A # send into figure (x)
  img_clustering = imadjust(img_clustering)

  # dilation (thing goes weird here)
  sed = matlab_strel_disk(7) # disk
  see = matlab_strel_ball(3,3) #circle 
  A = cv2.dilate(A,sed)
  # erosion
  A = cv2.erode(A, see)  
  # remove area < 800px 
  A = morphology.area_opening(A, area_threshold=800*3, connectivity=1) # vogado mention he use 800px
  img_morpho = A # send into figure (g)

  # debug mode 
  if(debug_mode):
    # resize image into width 200px 
    ratio = ncols/nrows
    t_width = 200
    t_height = math.ceil(t_width * ratio)
    print("(a) Original")
    cv2_imshow(resize_img(image_rgb, t_width))
    print("(b) M from CMYK")
    cv2_imshow(resize_img(cmyk_m, t_width))
    print("(c) *b from CIELAB")
    cv2_imshow(resize_img(lab_y, t_width))
    print("(d) M con adj + med(7x7)")
    cv2_imshow(resize_img(cmyk_m_con_med, t_width))
    print("(e) *b con adj + med(7x7)")
    cv2_imshow(resize_img(lab_y_con_med, t_width))
    print("(f) *b - M")
    cv2_imshow(resize_img(img_subt, t_width))
    print("(x) clustering")
    cv2_imshow(resize_img(img_clustering, t_width))
    print("(g)  Morphological Ops.")
    cv2_imshow(resize_img(img_morpho, t_width))

  print("--- %s seconds ---" % (time.time() - start_time))
  return img_morpho

In [ ]:
# SET THIS 
# path = folder contain image dataset and groundtruth
# gt_path = ground truth folder inside active dir
# data = data folder inside active dir
path = "drive/My Drive/ALL-IDB2/"
gt_path = path + "gt/"
target_path = path + "data/"

# here the loop 
# scanFolder(target_path, <limit image tested ["all"/int]>, <verbose>)
files_path, in_folder, total_files, file_scanned = scanFolder(target_path, "all", 0)
for f in files_path:
  original_image = cv2.imread(f)

  # load grount truth 
  gt_file_path = gt_path + getfilename(f) + ".png"
  gt_image = cv2.imread(gt_file_path)

  # convert gt into binary 
  gray = cv2.cvtColor(gt_image, cv2.COLOR_BGR2GRAY)
  (thresh, gt_image) = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
  
  # calculte dc
  print("kmeans:")
  kmeans = wbc_vogado(original_image, False)
  kmeans_dc = dice_coefficient(gt_image, kmeans)

  print("fcm:")
  fcm = wbc_vogado_modified(original_image, False) 
  fcm_dc = dice_coefficient(gt_image, fcm)

  # show gt_image 
  # cv2_imshow(resize_img(original_image,400))
  # cv2_imshow(resize_img(gt_image,400))
  # cv2_imshow(resize_img(kmeans,400))
  # cv2_imshow(resize_img(fcm,400))

kmeans:
--- 3.528958797454834 seconds ---


,Pred. (1),Pred. (0)
Actu. (1),14948.0 (TP),2222.0 (FN)
Actu. (0),328.0 (FP),48551.0 (TN)


,Accuracy (A),Specificity (S),Precision (P),Recall (R),Kappa Index (J),"Dice coefficiet (DC),"
0,0.961392,0.99329,0.978528,0.870588,0.895934,0.854269


fcm:
--- 3.9479758739471436 seconds ---


,Pred. (1),Pred. (0)
Actu. (1),14886.0 (TP),2284.0 (FN)
Actu. (0),296.0 (FP),48583.0 (TN)


...

,Pred. (1),Pred. (0)
Actu. (1),10356.0 (TP),4003.0 (FN)
Actu. (0),168.0 (FP),51522.0 (TN)


,Accuracy (A),Specificity (S),Precision (P),Recall (R),Kappa Index (J),"Dice coefficiet (DC),"
0,0.93685,0.99675,0.984036,0.72122,0.794605,0.712879


fcm:
--- 4.397639036178589 seconds ---


,Pred. (1),Pred. (0)
Actu. (1),9894.0 (TP),4465.0 (FN)
Actu. (0),137.0 (FP),51553.0 (TN)


,Accuracy (A),Specificity (S),Precision (P),Recall (R),Kappa Index (J),"Dice coefficiet (DC),"
0,0.930324,0.99735,0.986342,0.689045,0.770228,0.682533
